#**GRADIO UI**

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

In [ ]:
densenet_model = tf.keras.models.load_model("densenet_model.h5")
efficientnet_model = tf.keras.models.load_model("efficientnet_model.h5")

In [ ]:
class_names = ['No Cataract', 'Cataract']
IMG_SIZE = (224, 224)

In [ ]:
def preprocess(img, preprocess_func):
    img = img.resize(IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_func(img_array)
    return img_array

In [ ]:
def predict(img):

    densenet_input = preprocess(img, densenet_preprocess)
    efficientnet_input = preprocess(img, efficientnet_preprocess)

    densenet_pred = densenet_model.predict(densenet_input)[0][0]
    efficientnet_pred = efficientnet_model.predict(efficientnet_input)[0][0]

    densenet_label = class_names[int(densenet_pred >= 0.5)]
    efficientnet_label = class_names[int(efficientnet_pred >= 0.5)]

    densenet_result = {
        "Prediction": densenet_label,
        "Confidence": float(densenet_pred if densenet_pred >= 0.5 else 1 - densenet_pred)
    }

    efficientnet_result = {
        "Prediction": efficientnet_label,
        "Confidence": float(efficientnet_pred if efficientnet_pred >= 0.5 else 1 - efficientnet_pred)
    }

    return densenet_result, efficientnet_result

In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(label="DenseNet Result"),
        gr.Label(label="EfficientNet Result")
    ],
    title="Cataract Detection: DenseNet vs EfficientNet",
    description="Upload an eye image to check for cataract using two deep learning models."
)

In [ ]:
interface.launch(share=True)